---
layout: post
title:  Scala 2 Functional Programming
date:   2025-10-24
categories: [Scala]
mermaid: true
maths: true
typora-root-url: /Users/ojitha/GitHub/ojitha.github.io
typora-copy-images-to: ../assets/images/${filename}
---

<style>
/* Styles for the two-column layout */
.image-text-container {
    display: flex; /* Enables flexbox */
    flex-wrap: wrap; /* Allows columns to stack on small screens */
    gap: 20px; /* Space between the image and text */
    align-items: center; /* Vertically centers content in columns */
    margin-bottom: 20px; /* Space below this section */
}

.image-column {
    flex: 1; /* Allows this column to grow */
    min-width: 250px; /* Minimum width for the image column before stacking */
    max-width: 40%; /* Maximum width for the image column to not take up too much space initially */
    box-sizing: border-box; /* Include padding/border in element's total width/height */
}

.text-column {
    flex: 2; /* Allows this column to grow more (e.g., twice as much as image-column) */
    min-width: 300px; /* Minimum width for the text column before stacking */
    box-sizing: border-box;
}

</style>

<div class="image-text-container">
    <div class="image-column">
        <img src="/assets/images/2025-07-25-Scala-basics/scala_basics.png" alt="Scala basics" width="150" height="150">
    </div>
    <div class="text-column">
<p>TBC</p>
    </div>
</div>

<!--more-->

------

* TOC
{:toc}
------

Functional Programming (FP) is a programming paradigm based on mathematical functions where computation is treated as the evaluation of mathematical functions avoiding changing state and mutable data. This guide explores five fundamental FP concepts in Scala 2 with their mathematical foundations.

Core Principles of Functional Programming

1. **Functions are first-class values**: Functions can be passed as arguments, returned from other functions, and assigned to variables
2. **Immutability**: Data structures don't change after creation
3. **No side effects**: Pure functions don't modify external state
4. **Referential Transparency**: Expressions can be replaced by their values without changing program behavior

## Pure Functions

### Mathematical Definition

A **pure function** is a mathematical function where:

$$f: A \rightarrow B$$

Such that:

1. **Determinism**: $\forall x \in A, f(x)$ always produces the same output $y \in B$
2. **No Side Effects**: $f$ doesn't modify any state outside its scope
3. **Referential Transparency**: For any program $p$, all occurrences of $f(x)$ can be replaced by its result without changing $p$'s meaning

### Formal Definition

**Referential Transparency (RT)**: An expression $e$ is referentially transparent if, for all programs $p$, all occurrences of $e$ in $p$ can be replaced by the result of evaluating $e$ without affecting the meaning of $p$.[^1]

**Purity**: A function $f$ is pure if the expression $f(x)$ is referentially transparent for all referentially transparent $x$.[^1]

### Mathematical Properties

```mermaid
---
config:
  look: neo
  theme: default
---
graph TB
    A[Pure Function Properties] --> B[Determinism]
    A --> C[No Side Effects]
    A --> D[Referential Transparency]
    B --> E["∀x: f(x) = constant"]
    C --> F["No I/O, mutations, or exceptions"]
    D --> G["Can substitute f(x) with its result"]
    style A fill:#e1f5ff
    style B fill:#fff4e1
    style C fill:#fff4e1
    style D fill:#fff4e1
```

### Substitution Model

Pure functions enable **equational reasoning** through the substitution model:

$$
\begin{align}
\text{Given: } & f(x) = x^2 \\
\text{Then: } & f(3) + f(3) \\
& = 9 + f(3) \quad \text{(substitute first)} \\
& = 9 + 9 \quad \text{(substitute second)} \\
& = 18
\end{align}
$$

### Code Examples


In [6]:
// Pure Function - Always returns the same output for the same input
def square(x: Int): Int = x * x

// Test purity
val result1 = square(5)  // 25
val result2 = square(5)  // 25
// result1 == result2 always true

defined function square
result1: Int = 25
result2: Int = 25

Mathematical proof:
    
    square(5) = 5 * 5 = 25

NOTE: Can always replace square(5) with 25


In [7]:
// Impure Function - Has side effects
class CreditCard {
  def charge(amount: Double): Unit = {
    println(s"Charging $amount")  // Side effect: I/O
    // External API call would go here
  }
}

def buyCoffee(cc: CreditCard, price: Double): Unit = {
  cc.charge(price)  // Side effect!
  println("Coffee purchased")
}


defined class CreditCard
defined function buyCoffee

Problem: Cannot substitute buyCoffee(cc, 2.50) with its "result" because the function performs actions, not just computations

Making Impure Code Pure

In [8]:
// Impure version
def buyCoffeeImpure(cc: CreditCard): Coffee = {
  val cup = new Coffee()
  cc.charge(cup.price)  // Side effect
  cup
}

// Pure version - Return description of charge
case class Coffee(price: Double = 2.50)
case class Charge(cc: CreditCard, amount: Double)

def buyCoffeePure(cc: CreditCard): (Coffee, Charge) = {
  val cup = new Coffee()
  (cup, Charge(cc, cup.price))  // No side effects!
}


defined function buyCoffeeImpure
defined class Coffee
defined class Charge
defined function buyCoffeePure

Mathematical property:
 buyCoffeePure(cc) can be replaced by (Coffee(2.50), Charge(cc, 2.50))

Pure Function with Local Mutation


In [9]:
// Pure function using internal mutable state (hidden from outside)
def quicksort(xs: List[Int]): List[Int] = {
  if (xs.isEmpty) xs
  else {
    val arr = xs.toArray  // Local mutable array 👈
    
    def swap(x: Int, y: Int): Unit = {
      val tmp = arr(x)
      arr(x) = arr(y)
      arr(y) = tmp
    }
    
    def partition(n: Int, r: Int, pivot: Int): Int = {
      val pivotVal = arr(pivot)
      swap(pivot, r)
      var j = n
      for (i <- n until r) {
        if (arr(i) < pivotVal) {
          swap(i, j)
          j += 1
        }
      }
      swap(j, r)
      j
    }
    
    def qs(n: Int, r: Int): Unit = {
      if (n < r) {
        val pi = partition(n, r, n + (r - n) / 2)
        qs(n, pi - 1)
        qs(pi + 1, r)
      }
    }
    
    qs(0, arr.length - 1)
    arr.toList
  }
}



defined function quicksort

Still pure because:
1. Local mutations not observable from outside
2. For any List xs, quicksort(xs) always returns the same sorted list
3. Can replace `quicksort(List(3,1,2))` with `List(1,2,3)`

### Pure Functions in Mathematics

| Mathematical Concept | Scala Example                      | Property           |
| -------------------- | ---------------------------------- | ------------------ |
| $f(x) = x + 1$       | `def inc(x: Int) = x + 1`          | Successor function |
| $f(x, y) = x + y$    | `def add(x: Int, y: Int) = x + y`  | Addition           |
| $f(x) = x^2$         | `def square(x: Int) = x * x`       | Quadratic          |
| $f(s) = \|s\|$       | `def length(s: String) = s.length` | String length      |
| $f(x) = e^x$         | `def exp(x: Double) = math.exp(x)` | Exponential        |

### Key Benefits of Pure Functions

1. **Testability**: Easy to test - same input always produces the same output
2. **Parallelisation**: Can safely run in parallel (no shared mutable state)
3. **Memoization**: Results can be cached
4. **Reasoning**: Enable algebraic reasoning and equational substitution
5. **Composition**: Can combine pure functions to create complex behaviour

```mermaid
---
config:
  look: neo
  theme: default
---
mindmap
  root((Pure Functions))
    Benefits
      Easy Testing
        Predictable
        No Mocks Needed
      Parallelization
        Thread Safe
        No Race Conditions
      Caching
        Memoization
        Performance
    Properties
      Deterministic
      No Side Effects
      Referential Transparency
    Applications
      Mathematical Computations
      Data Transformations
      Functional Pipelines
```

---

## Higher-Order Functions

### Mathematical Definition

A **higher-order function (HOF)** is a function that:

1. Takes one or more functions as arguments, or
2. Returns a function as its result

$$
\text{HOF}: (A \rightarrow B) \rightarrow C \text{ or } A \rightarrow (B \rightarrow C)
$$

> In category theory, *HOFs are morphisms* in the category of functions where *objects are types and morphisms are functions*.[^4]

### Type Theory Foundation

In the simply typed lambda calculus, if $\tau_1, \tau_2, \tau_3$ are types:

$$
f: (\tau_1 \rightarrow \tau_2) \rightarrow \tau_3 \text{ is a HOF}
$$

This means $f$ accepts a function of type $\tau_1 \rightarrow \tau_2$ and produces a value of type $\tau_3$.

Examples

1. Map operation (Functor)
2. Filter operation
3. Fold operation:

#### Example 1: Map - The Functor HOF[^2]

**Map operation (Functor)**:

$$
\text{map}: (A \rightarrow B) \rightarrow [A] \rightarrow [B]
$$


>This reads as: "`map` takes a function from A to B, then takes a list of A's, and returns a list of B's".
{:.info-box}

The arrows represent **currying** - `map` doesn't take all arguments at once, but returns functions:
-   Takes `(A → B)` - a function from type A to type B
-   Returns a function that takes `[A]` - a list of A's
-   Which returns `[B]` - a list of B's

**Behaviour:**

$$
\text{map}\ f\ [x_1, x_2, ..., x_n] = [f(x_1), f(x_2), ..., f(x_n)]
$$
    
Apply function f to each element in the list.

The map applies a function to each element

In [ ]:
def map[A, B](f: A => B)(xs: List[A]): List[B] = xs match {
  case Nil => Nil
  case head :: tail => f(head) :: map(f)(tail)
}

**Type Parameters:** `[A, B]` - generic types matching the math notation

**Curried Syntax:** Two parameter lists `(f: A => B)(xs: List[A])` implements the curried arrows:

-   First takes function `f: A => B` (matching `A → B`)
-   Then takes list `xs: List[A]` (matching `[A]`)
-   Returns `List[B]` (matching `[B]`)

The recursive case implements the mathematical definition:

-   `f(head)` - apply f to first element (like `f(x₁)`)
-   `map(f)(tail)` - recursively map over remaining elements (like `[f(x₂), ..., f(xₙ)]`)
-   `::` - cons operator joins them together

Example execution is
```scala
map(x => x * 2)(List(1, 2, 3))
```

Traces to:
```scala
f(1) :: map(f)(List(2, 3))
2 :: f(2) :: map(f)(List(3))
2 :: 4 :: f(3) :: map(f)(Nil)
2 :: 4 :: 6 :: Nil
List(2, 4, 6)
```

This matches the math: `map (*2) [1,2,3] = [2, 4, 6]` ✓

In [13]:
// Using map
val numbers = List(1, 2, 3)
val squared = map((x: Int) => x * 2)(numbers)
// Result: List(1, 4, 9, 16, 25)

// Mathematical property (Functor law):
// map(id) == id
// map(f ∘ g) == map(f) ∘ map(g)

numbers: List[Int] = List(1, 2, 3)
squared: List[Int] = List(2, 4, 6)

Scala standard library usage

In [15]:
val nums = List(1, 2, 3)

// map with named function
def double(x: Int): Int = x * 2
nums.map(double)  // List(2, 4, 6, 8, 10)

nums: List[Int] = List(1, 2, 3)
defined function double
res14_2: List[Int] = List(2, 4, 6)

#### Example 2: Filter - Predicate-Based Selection[^2]

filter keeps elements satisfying a predicate

$$
\text{filter}: (A \rightarrow \text{Bool}) \rightarrow [A] \rightarrow [A]
$$
    
> This reads as: "`filter` takes a predicate function from A to Boolean, then takes a list of A's, and returns a list of A's"
{:.info-box}

Key difference from `map`: the input and output list types are **the same** (`[A] → [A]`) because we're selecting elements, not transforming them.

The currying structure:

-   Takes `(A → Bool)` - a predicate function that tests elements
-   Returns a function that takes `[A]` - a list of A's
-   Which returns `[A]` - a filtered list of A's

**Behaviour (List Comprehension)**:

This is **list comprehension** notation meaning: "build a list containing all elements `x` drawn from `xs` where the predicate `p(x)` is true"

$$
\text{filter}\ p\ xs = [x\ |\ x \leftarrow xs, p(x)]
$$
    

In [16]:
def filter[A](p: A => Boolean)(xs: List[A]): List[A] = xs match {
  case Nil => Nil
  case head :: tail => 
    if (p(head)) head :: filter(p)(tail)
    else filter(p)(tail)
}

defined function filter

**Type Parameter:** `[A]` - single generic type (input and output are same type)

**Curried Syntax:** `(p: A => Boolean)(xs: List[A])` implements:

-   First takes predicate `p: A => Boolean` (matching `A → Bool`)
-   Then takes list `xs: List[A]` (matching `[A]`)
-   Returns `List[A]` (matching `[A]`)

**Pattern Matching Implementation:**

The recursive logic implements the list comprehension:

-   **Test:** `if (p(head))` - checks if predicate holds for current element
-   **Include:** `head :: filter(p)(tail)` - keep element and continue
-   **Exclude:** `filter(p)(tail)` - skip element and continue
-   Eventually builds a list of only elements where `p(x)` was true


In [17]:
// Using filter
val numbers = List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

// Keep even numbers
val evens = filter((x: Int) => x % 2 == 0)(numbers)
// Result: List(2, 4, 6, 8, 10)

// Mathematical definition as list comprehension:
// filter p xs = [x | x ← xs, p x]

numbers: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
evens: List[Int] = List(2, 4, 6, 8, 10)

Example Execution
```scala
filter(x => x > 2)(List(1, 2, 3, 4))
```

Traces to:
```scala
p(1) = false → filter(p)(List(2, 3, 4))
p(2) = false → filter(p)(List(3, 4))
p(3) = true  → 3 :: filter(p)(List(4))
p(4) = true  → 3 :: 4 :: filter(p)(Nil)
3 :: 4 :: Nil
List(3, 4)
```

Scala Standard library usage:

In [19]:
// Practical examples
val numbers = List(-3, -2, -1, 0, 1, 2, 3, 4, 5)

// Filter positive numbers
numbers.filter(_ > 0)  // List(1, 2, 3)

// Filter even numbers
numbers.filter(x => x % 2 == 0)  // List(-2, 0, 2, 4)

// Chain filter operations
numbers
  .filter(_ > 0)        // List(1, 2, 3, 4, 5)
  .filter(_ % 2 == 0)   // List(2, 4)

numbers: List[Int] = List(-3, -2, -1, 0, 1, 2, 3, 4, 5)
res18_1: List[Int] = List(1, 2, 3, 4, 5)
res18_2: List[Int] = List(-2, 0, 2, 4)
res18_3: List[Int] = List(2, 4)

This matches the math: `map (*2) [1,2,3] = [2, 4, 6]` ✓

#### Example 3: Fold - Reduction Operations[^2]

$$
\text{foldr}: (A \rightarrow B \rightarrow B) \rightarrow B \rightarrow [A] \rightarrow B
$$

> This reads as: "`foldr` takes a combining function, then an initial value, then a list, and returns a single accumulated value"
{:.info-box}    

This reads as: "`foldr` takes a combining function, then an initial value, then a list, and returns a single accumulated value"

The currying structure (three levels):

-   Takes `(A → B → B)` - a **combining function** that takes an element of type A and an accumulator of type B, returning a new B
-   Then takes `B` - an **initial/accumulator value**
-   Then takes `[A]` - a list of A's
-   Returns `B` - a single folded result

**Behaviour (Right-Associative):**

$$
\text{foldr}\ (\oplus)\ a\ [x_1, x_2, ..., x_n] = x_1 \oplus (x_2 \oplus ... (x_n \oplus a))
$$

Works right-to-left, with parentheses showing association from the right. The rightmost element xₙ is combined with the initial value first, then that result is combined with xₙ₋₁, and so on.

foldr - right-associative fold

In [20]:
def foldr[A, B](f: (A, B) => B)(z: B)(xs: List[A]): B = xs match {
  case Nil => z
  case head :: tail => f(head, foldr(f)(z)(tail))
}

// Mathematical notation:
// foldr (⊕) a [x₁, x₂, ..., xₙ] = x₁ ⊕ (x₂ ⊕ ... (xₙ ⊕ a))

defined function foldr

**Type Parameters:** `[A, B]` - two types (list elements and accumulator can differ)

**Three Parameter Lists (Curried):**

-   `(f: (A, B) => B)` - combining function (matching `A → B → B`)
-   `(z: B)` - initial/"zero" value (matching the `B`)
-   `(xs: List[A])` - input list (matching `[A]`)
-   Returns `B` - folded result


The recursive case implements right-associativity:

-   `foldr(f)(z)(tail)` - **first** recursively fold the tail (processes right side)
-   `f(head, ...)` - **then** combine head with that result (processes left side)
-   This creates the structure: `head ⊕ (recursive_result)`

Example Execution


In [23]:
// Practical fold examples
val numbers = List(1, 2, 3, 4, 5)
// Product using foldRight
val product = numbers.foldRight(1)(_ * _)  // 120
// Evaluation: 1 * (2 * (3 * (4 * (5 * 1))))

numbers: List[Int] = List(1, 2, 3, 4, 5)
product: Int = 120

```scala
foldr((x: Int, acc: Int) => x + acc)(0)(List(1, 2, 3))
```

Traces to:
```
f(1, foldr(f)(0)(List(2, 3)))
f(1, f(2, foldr(f)(0)(List(3))))
f(1, f(2, f(3, foldr(f)(0)(Nil))))
f(1, f(2, f(3, 0)))
f(1, f(2, 3))
f(1, 5)
6
```

This matches the math: 
$$\text{foldr}\ (+)\ 0\ [1,2,3] = 1 + (2 + (3 + 0)) = 1 + (2 + 3) = 1 + 5 = 6$$ ✓

Visualising Right Association

For `foldr (⊕) a [x₁, x₂, x₃]`:
```
    x₁ ⊕ (x₂ ⊕ (x₃ ⊕ a))
    │      │      │
    │      │      └─ combines first
    │      └──────── combines second  
    └─────────────── combines last
```

foldl - left-associative fold

$$
\text{foldl}: (B \rightarrow A \rightarrow B) \rightarrow B \rightarrow [A] \rightarrow B
$$

> This reads as: "`foldl` takes a combining function, then an initial value, then a list, and returns a single accumulated value"
{:.info-box}

The currying structure (three levels):
- Takes `(B → A → B)` - a **combining function** that takes an accumulator of type B and an element of type A, returning a new B
  - **Note:** Arguments are **reversed** compared to `foldr`!
- Then takes `B` - an **initial/accumulator value**
- Then takes `[A]` - a list of A's
- Returns `B` - a single folded result

**Behaviour (Left-Associative):**

$$
\text{foldl}\ (\oplus)\ a\ [x_1, x_2, ..., x_n] = ((...((a \oplus x_1) \oplus x_2) ...) \oplus x_n)
$$

Works **left-to-right**, with parentheses showing association from the left. The leftmost element `x₁` is combined with the initial value `a` first, then that result is combined with `x₂`, and so on.


In [25]:
// foldl - left-associative fold
def foldl[A, B](f: (B, A) => B)(z: B)(xs: List[A]): B = xs match {
  case Nil => z
  case head :: tail => foldl(f)(f(z, head))(tail)
}

// Mathematical notation:
// foldl (⊕) a [x₁, x₂, ..., xₙ] = (((a ⊕ x₁) ⊕ x₂) ... ⊕ xₙ)

defined function foldl

**Type Parameters:** `[A, B]` - two types (list elements and accumulator can differ)

**Three Parameter Lists (Curried):**
- `(f: (B, A) => B)` - combining function with **accumulator first** (matching `B → A → B`)
- `(z: B)` - initial/"zero" value (matching the `B`)
- `(xs: List[A])` - input list (matching `[A]`)
- Returns `B` - folded result


In [26]:
// Building a reversed list with foldLeft
val reversed = numbers.foldLeft(List.empty[Int])((acc, x) => x :: acc)
// Result: List(5, 4, 3, 2, 1)

reversed: List[Int] = List(5, 4, 3, 2, 1)

Example Execution

The recursive case implements left-associativity:
- `f(z, head)` - **first** combine accumulator with head (processes left side)
- `foldl(f)(...)(tail)` - **then** recursively fold tail with new accumulator
- This creates the structure: `(previous_result) ⊕ head`

```scala
foldl((acc: Int, x: Int) => acc + x)(0)(List(1, 2, 3))
```

Traces to:
```
foldl(f)(f(0, 1))(List(2, 3))
foldl(f)(1)(List(2, 3))
foldl(f)(f(1, 2))(List(3))
foldl(f)(3)(List(3))
foldl(f)(f(3, 3))(Nil)
foldl(f)(6)(Nil)
6
```

This matches the math:
$$\text{foldl}\ (+)\ 0\ [1,2,3] = ((0 + 1) + 2) + 3 = (1 + 2) + 3 = 3 + 3 = 6$$ ✓

## Visualising Left Association

For `foldl (⊕) a [x₁, x₂, x₃]`:

```
    ((a ⊕ x₁) ⊕ x₂) ⊕ x₃
     │         │       │
     │         │       └─ combines last
     │         └──────── combines second  
     └─────────────────── combines first
```


Scala Standard library usage:

In [27]:
// Practical fold examples
val numbers = List(1, 2, 3, 4, 5)

// Sum using foldLeft
val sum = numbers.foldLeft(0)(_ + _)  // 15
// Evaluation: ((((0 + 1) + 2) + 3) + 4) + 5

// Product using foldRight
val product = numbers.foldRight(1)(_ * _)  // 120
// Evaluation: 1 * (2 * (3 * (4 * (5 * 1))))

// Building a reversed list with foldLeft
val reversed = numbers.foldLeft(List.empty[Int])((acc, x) => x :: acc)
// Result: List(5, 4, 3, 2, 1)

numbers: List[Int] = List(1, 2, 3, 4, 5)
sum: Int = 15
product: Int = 120
reversed: List[Int] = List(5, 4, 3, 2, 1)

Defining common operations with fold

In [28]:
def sum(xs: List[Int]): Int = 
  xs.foldRight(0)(_ + _)

def product(xs: List[Int]): Int = 
  xs.foldRight(1)(_ * _)

def length[A](xs: List[A]): Int = 
  xs.foldRight(0)((_, acc) => acc + 1)

def concatenate[A](xss: List[List[A]]): List[A] = 
  xss.foldRight(List.empty[A])(_ ++ _)

// Example usage
sum(List(1, 2, 3, 4, 5))           // 15
product(List(1, 2, 3, 4, 5))       // 120
length(List('a', 'b', 'c'))        // 3
concatenate(List(List(1,2), List(3,4)))  // List(1, 2, 3, 4)

defined function sum
defined function product
defined function length
defined function concatenate
res27_4: Int = 15
res27_5: Int = 120
res27_6: Int = 3
res27_7: List[Int] = List(1, 2, 3, 4)

#### Comparing foldr vs foldl

##### Visual Comparison

**foldr:** `x₁ ⊕ (x₂ ⊕ (x₃ ⊕ a))`  
- Right-associative
- Processes right side first (via recursion)
- Function signature: `(A, B) => B`

**foldl:** `((a ⊕ x₁) ⊕ x₂) ⊕ x₃`  
- Left-associative  
- Processes left side first (via accumulator)
- Function signature: `(B, A) => B`

##### Example Where Order Matters

```scala
// Division is NOT associative
foldr((x: Int, acc: Int) => x / acc)(1)(List(8, 4, 2))
// 8 / (4 / (2 / 1)) = 8 / (4 / 2) = 8 / 2 = 4

foldl((acc: Int, x: Int) => acc / x)(64)(List(4, 2, 2))
// ((64 / 4) / 2) / 2 = (16 / 2) / 2 = 8 / 2 = 4
```

Different structure, potentially different results!

##### Example: Building a Reversed List

```scala
foldl((acc: List[Int], x: Int) => x :: acc)(Nil)(List(1, 2, 3))
```

```
((Nil :: 1) :: 2) :: 3
(List(1) :: 2) :: 3
List(2, 1) :: 3
List(3, 2, 1)
```

This **reverses** the list! In fact, `foldl (flip (::)) Nil xs = reverse xs`

#### Key Insights

- **`foldl` is tail-recursive** in this implementation - the recursive call is the last operation, making it more memory efficient
- **Left-associative:** processes list from left to right via accumulator
- **Accumulator-first:** the function takes `(B, A)` not `(A, B)` - accumulator is first argument
- **For associative operations** (like `+`, `*`), `foldr` and `foldl` produce the same result
- **For non-associative operations** (like `-`, `/`, `::`) they produce different results
- **In practice:** `foldl` is often preferred for efficiency (tail-recursive), but `foldr` works on infinite lists

[^1]: Functional Programming in Scala, Ch. 1: "What is functional programming?" → "The benefits of FP: a simple example", p. 3-12

[^2]: Bird & Wadler: Introduction to Functional Programming, Ch. 3: "Lists" → "Map and filter", p. 61-65

[^3]: Programming in Scala Fourth Edition, Ch. 23: "For Expressions Revisited" → p. 528-545

[^4]: Category Theory, Ch. 9: "Functors and Natural Transformations" → p. 156-161

[^5]: Scala in Depth, Ch. 11: "Patterns in functional programming" → "Currying and applicative style", p. 266-268

[^6]: Programming in Scala Fourth Edition, Ch. 8: "Functions and Closures" → "Closures", p. 176-179

{:gtxt: .message color="green"}
{:ytxt: .message color="yellow"}
{:rtxt: .message color="red"}